
### 🕵️‍♂️ Script: Caçador de Anomalias de Preço

> **Fase:** Validação de Dados / Sanity Check
> **Arquivo Alvo:** `aurum_master_features.parquet`

Este script é uma ferramenta de **Auditoria de Dados** projetada para identificar e isolar erros graves na base histórica de preços, especificamente falhas no ajuste de eventos corporativos (Grupamentos/Inplits e Desdobramentos/Splits).

### 🎯 Objetivo

Evitar que o Backtest apresente lucros falsos ou irreais (ex: 900% em um mês) causados por dados sujos. Ele atua como um "filtro de sanidade" antes que os dados sejam consumidos pelo motor de estratégia.

### ⚙️ Como Funciona

O script carrega o dataset mestre e calcula a variação percentual mensal (`pct_change`) de cada ativo. Ele então sinaliza qualquer movimentação que viole os limites da realidade estatística de uma "Blue Chip" ou ativo líquido do IBRX-100.

#### 📐 Parâmetros de Corte (Thresholds)

| Parâmetro | Valor | Descrição | Causa Provável do Erro |
| --- | --- | --- | --- |
| **LIMITE_ALTA** | `2.0` (+200%) | Variação positiva máxima tolerada em 1 mês. | **Grupamento (Inplit)** não ajustado. O preço salta de R$ 1 para R$ 10 artificialmente. |
| **LIMITE_BAIXA** | `-0.9` (-90%) | Variação negativa máxima tolerada em 1 mês. | **Desdobramento (Split)** não ajustado ou falência. O preço cai de R$ 10 para R$ 1 artificialmente. |

### 🛠️ Estrutura do Script

#### 1. Leitura e Preparação

Carrega o arquivo `.parquet` unificado e garante que as datas estejam ordenadas cronologicamente para o cálculo correto dos retornos.

#### 2. Cálculo de Retorno

```python
df['retorno_mensal'] = df.groupby('ticker')['Adj Close'].pct_change()

```

Calcula a variação do preço de fechamento ajustado mês a mês.

#### 3. Detecção

Filtra o DataFrame buscando linhas onde:
`retorno > 200%` **OU** `retorno < -90%`

#### 4. Verificação Específica (Debug)

O script possui um bloco de verificação manual para o período de **Dezembro/2021** focando em ativos específicos (`USIM5`, `GGBR4`, `CSNA3`, `PETR4`, `PETR3`). Isso serve para validar se correções anteriores surtiram efeito.

#### 📂 Outputs (Saídas)

1. **Console (Terminal):**
* Exibe um alerta vermelho 🚨 se anomalias forem encontradas.
* Imprime uma tabela com as piores variações para inspeção rápida.


2. **Arquivo CSV:**
* Se anomalias forem detectadas, salva o relatório detalhado em:
* `../data/anomalias/anomalias_encontradas.csv`



#### 🚀 Como Executar

Certifique-se de que o arquivo mestre (`aurum_master_features.parquet`) existe no diretório correto.

```bash
python src/step_03_cacar_anomalias.py

```

#### 📋 Exemplo de Resultado (Anomalia Detectada)

Se o script encontrar um erro de grupamento na `MGLU3`, o output será similar a:

```text
🚨 ENCONTRADAS 1 ANOMALIAS POTENCIAIS!
Estas ações tiveram variações de preço irreais:
------------------------------------------------------------
             date    ticker  Adj Close  retorno_mensal
1542   2023-05-31  MGLU3.SA      45.00        9.000000
------------------------------------------------------------
💾 Lista salva em 'anomalias_encontradas.csv'

```

*(Neste exemplo hipotético, um retorno de 9.0 (900%) indica um erro de dados, pois a ação não subiu isso de verdade).*

---

In [1]:
import pandas as pd
import numpy as np

FILE_PATH = "../data/aurum_master_features.parquet"

def cacar_anomalias():
    print(f"🔍 Lendo arquivo: {FILE_PATH}...")
    try:
        df = pd.read_parquet(FILE_PATH)
    except FileNotFoundError:
        print("❌ Arquivo não encontrado.")
        return

    df['date'] = pd.to_datetime(df['date'])
    df = df.sort_values(['ticker', 'date'])

    df['retorno_mensal'] = df.groupby('ticker')['Adj Close'].pct_change()

    LIMITE_ALTA = 2.0  
    LIMITE_BAIXA = -0.9

    anomalias = df[ (df['retorno_mensal'] > LIMITE_ALTA) | (df['retorno_mensal'] < LIMITE_BAIXA) ]

    if anomalias.empty:
        print("✅ Nenhuma anomalia grave encontrada (Retornos entre -90% e +200%).")
        print("Vamos verificar o Top 5 de Dezembro de 2021 manualmente...")
        # Filtro específico para o mês do problema
        mask_dec21 = (df['date'] == '2021-12-01') | (df['date'] == '2021-12-31')
        top_suspeitos = ['USIM5.SA', 'GGBR4.SA', 'CSNA3.SA', 'PETR4.SA', 'PETR3.SA']
        print(df[mask_dec21 & df['ticker'].isin(top_suspeitos)][['date', 'ticker', 'Adj Close']])
    else:
        print(f"\n🚨 ENCONTRADAS {len(anomalias)} ANOMALIAS POTENCIAIS!")
        print("Estas ações tiveram variações de preço irreais:")
        print("-" * 60)
        # Mostra as piores anomalias
        print(anomalias[['date', 'ticker', 'Adj Close', 'retorno_mensal']].sort_values('retorno_mensal', ascending=False).to_string())
        print("-" * 60)
        
        # Salva para você olhar
        anomalias.to_csv("../data/anomalias/anomalias_encontradas.csv", sep=';')
        print("💾 Lista salva em 'anomalias_encontradas.csv'")

if __name__ == "__main__":
    cacar_anomalias()

🔍 Lendo arquivo: ../data/aurum_master_features.parquet...

🚨 ENCONTRADAS 5 ANOMALIAS POTENCIAIS!
Estas ações tiveram variações de preço irreais:
------------------------------------------------------------
            date    ticker  Adj Close  retorno_mensal
6170  2017-09-30  MGLU3.SA  20.644400       16.162748
11177 2021-06-30  CPLE5.SA  39.987400        8.652573
7160  2018-06-30  SLCE3.SA   7.924632        3.187190
9688  2020-06-30  PRIO3.SA   7.066650        2.655992
10825 2021-03-31  PCAR3.SA  31.085117        2.019108
------------------------------------------------------------
💾 Lista salva em 'anomalias_encontradas.csv'
